In [ ]:
import os
os.environ['KAGGLE_USERNAME']="mqasimali"
os.environ['KAGGLE_KEY']="03318089e106031ba1a9847ff6f1fd25"
!kaggle datasets download tommyngx/digital-knee-xray
!unzip /content/digital-knee-xray

Dataset URL: https://www.kaggle.com/datasets/tommyngx/digital-knee-xray
License(s): Attribution 4.0 International (CC BY 4.0)
 92% 107M/117M [00:01<00:00, 101MB/s]  
100% 117M/117M [00:01<00:00, 95.6MB/s]
Archive:  /content/digital-knee-xray.zip
  inflating: MedicalExpert-I/0Normal/NormalG0 (1).png  
  inflating: MedicalExpert-I/0Normal/NormalG0 (10).png  
  inflating: MedicalExpert-I/0Normal/NormalG0 (100).png  
  inflating: MedicalExpert-I/0Normal/NormalG0 (101).png  
  inflating: MedicalExpert-I/0Normal/NormalG0 (102).png  
  inflating: MedicalExpert-I/0Normal/NormalG0 (103).png  
  inflating: MedicalExpert-I/0Normal/NormalG0 (104).png  
  inflating: MedicalExpert-I/0Normal/NormalG0 (105).png  
  inflating: MedicalExpert-I/0Normal/NormalG0 (106).png  
  inflating: MedicalExpert-I/0Normal/NormalG0 (107).png  
  inflating: MedicalExpert-I/0Normal/NormalG0 (108).png  
  inflating: MedicalExpert-I/0Normal/NormalG0 (109).png  
  inflating: MedicalExpert-I/0Normal/NormalG0 (11).png  
  in

In [ ]:
import os
import numpy as np
import cv2
from PIL import Image
from collections import Counter
from sklearn.utils import resample

np.random.seed(1000)

image_directory = '/content/MedicalExpert-I/'
SIZE = 75

images = []
labels = []

class_names = ['Normal', 'Doubtful', 'Mild', 'Moderate', 'Severe']
class_labels = {class_name: idx for idx, class_name in enumerate(class_names)}

for class_name, label in class_labels.items():
    class_directory = os.path.join(image_directory, class_name)
    image_files = os.listdir(class_directory)

    for image_name in image_files:
        if image_name.endswith('.png'):
            image_path = os.path.join(class_directory, image_name)
            image = cv2.imread(image_path)

            image = Image.fromarray(image, 'RGB')
            image = image.resize((SIZE, SIZE))

            images.append(np.array(image))
            labels.append(label)

images = np.array(images)
labels = np.array(labels)

data = list(zip(images, labels))

class_counts = Counter(labels)
print("Class distribution before oversampling:", class_counts)

max_count = max(class_counts.values())

data_by_class = {label: [] for label in class_labels.values()}
for image, label in data:
    data_by_class[label].append(image)

oversampled_data = []
for label, images in data_by_class.items():
    if len(images) < max_count:
        oversampled_images = resample(images, replace=True, n_samples=max_count, random_state=1000)
    else:
        oversampled_images = images
    oversampled_data.extend([(img, label) for img in oversampled_images])

np.random.shuffle(oversampled_data)

oversampled_images, oversampled_labels = zip(*oversampled_data)

oversampled_images = np.array(oversampled_images)
oversampled_labels = np.array(oversampled_labels)

print("Class distribution after oversampling:", Counter(oversampled_labels))


Class distribution before oversampling: Counter({0: 514, 1: 477, 2: 232, 3: 221, 4: 206})
Class distribution after oversampling: Counter({0: 514, 3: 514, 1: 514, 2: 514, 4: 514})


In [ ]:
from sklearn.model_selection import train_test_split

train_images, test_images, train_labels, test_labels = train_test_split(
    oversampled_images, oversampled_labels, test_size=0.20, random_state=1000, stratify=oversampled_labels
)

train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.20, random_state=1000, stratify=train_labels
)

print(f"Training set size: {len(train_images)}")
print(f"Validation set size: {len(val_images)}")
print(f"Test set size: {len(test_images)}")


Training set size: 1644
Validation set size: 412
Test set size: 514


In [ ]:
# Normalize the images
train_images = train_images / 255.0
val_images = val_images / 255.0
test_images = test_images / 255.0


In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# # Define the model
# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(SIZE, SIZE, 3)),
#     MaxPooling2D((2, 2)),
#     Conv2D(64, (3, 3), activation='relu'),
#     MaxPooling2D((2, 2)),
#     Conv2D(128, (3, 3), activation='relu'),
#     MaxPooling2D((2, 2)),
#     Flatten(),
#     Dense(512, activation='relu'),
#     Dropout(0.5),
#     Dense(5, activation='softmax')  # 5 classes
# ])

# # Compile the model
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Train the model
# history = model.fit(train_images, train_labels, epochs=80, validation_data=(val_images, val_labels), batch_size=32)

# # Evaluate the model on the test set
# test_loss, test_accuracy = model.evaluate(test_images, test_labels)
# print(f"Test accuracy: {test_accuracy}")


Epoch 1/80
52/52 [==============================] - 2s 21ms/step - loss: 1.6069 - accuracy: 0.2257 - val_loss: 1.5146 - val_accuracy: 0.3447
Epoch 2/80
52/52 [==============================] - 1s 12ms/step - loss: 1.5373 - accuracy: 0.2956 - val_loss: 1.4819 - val_accuracy: 0.3252
Epoch 3/80
52/52 [==============================] - 1s 11ms/step - loss: 1.4803 - accuracy: 0.3370 - val_loss: 1.4180 - val_accuracy: 0.3956
Epoch 4/80
52/52 [==============================] - 1s 11ms/step - loss: 1.4425 - accuracy: 0.3637 - val_loss: 1.4417 - val_accuracy: 0.3422
Epoch 5/80
52/52 [==============================] - 1s 10ms/step - loss: 1.3788 - accuracy: 0.3923 - val_loss: 1.3290 - val_accuracy: 0.4393
Epoch 6/80
52/52 [==============================] - 1s 10ms/step - loss: 1.3126 - accuracy: 0.4282 - val_loss: 1.3406 - val_accuracy: 0.4199
Epoch 7/80
52/52 [==============================] - 1s 10ms/step - loss: 1.2347 - accuracy: 0.4872 - val_loss: 1.1850 - val_accuracy: 0.4757
Epoch 8/80
52

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')  
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

history = model.fit(
    train_images, train_labels,
    epochs=80,
    validation_data=(val_images, val_labels),
    batch_size=32,
    callbacks=[early_stopping]
)

test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_accuracy}")


Epoch 1/80
52/52 [==============================] - 8s 79ms/step - loss: 1.5711 - accuracy: 0.4404 - val_loss: 0.9925 - val_accuracy: 0.5995
Epoch 2/80
52/52 [==============================] - 1s 23ms/step - loss: 0.9577 - accuracy: 0.6235 - val_loss: 0.8445 - val_accuracy: 0.6723
Epoch 3/80
52/52 [==============================] - 1s 26ms/step - loss: 0.7429 - accuracy: 0.7269 - val_loss: 0.7036 - val_accuracy: 0.7500
Epoch 4/80
52/52 [==============================] - 1s 22ms/step - loss: 0.5965 - accuracy: 0.7968 - val_loss: 0.6175 - val_accuracy: 0.7937
Epoch 5/80
52/52 [==============================] - 1s 22ms/step - loss: 0.4637 - accuracy: 0.8491 - val_loss: 0.5114 - val_accuracy: 0.8301
Epoch 6/80
52/52 [==============================] - 1s 22ms/step - loss: 0.3818 - accuracy: 0.8753 - val_loss: 0.4742 - val_accuracy: 0.8350
Epoch 7/80
52/52 [==============================] - 1s 22ms/step - loss: 0.3257 - accuracy: 0.8923 - val_loss: 0.5139 - val_accuracy: 0.8083
Epoch 8/80
52

In [ ]:
model.save("knee.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
